In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
%config InlineBackend.figure_format='retina'
from datetime import datetime
import numpy as np

In [ ]:
df_master = pd.read_csv('Data/master_data.csv')

In [ ]:
df_master.info()

In [ ]:
df = df_master.copy()

In [ ]:
df.columns[1:15]

In [ ]:
df['COUNTRY'].value_counts()

In [ ]:
df['Days_last_order'] = (datetime.now() - pd.to_datetime(df['CREATED_AT'])).dt.days

In [ ]:
df.drop(['Unnamed: 0', 'ID_x', 'FIRST_NAME_x', 'ID', 'IS_DEFAULT', 'NAME', 'FIRST_NAME_x',
               'PHONE_y', 'ADDRESS_1', 'ADDRESS_2', 'LAST_NAME_x', 'COMPANY', 'FIRST_NAME_y', 'LAST_NAME_y',
               'ACCEPTS_MARKETING_UPDATED_AT', 'CREATED_AT', 'EMAIL', 'LAST_NAME_x', 'MARKETING_OPT_IN_LEVEL',
               'NOTE', 'PHONE_x', 'STATE', 'TAX_EXEMPT', 'UPDATED_AT', 'VERIFIED_EMAIL', 'DEFAULT_ADDRESS_ID',
               '_FIVETRAN_SYNCED_x', '_FIVETRAN_SYNCED_y', 'CAN_DELETE', 'MULTIPASS_IDENTIFIER', 'COUNT(USER_ID)', 
               'SUM(TOTAL_SPENT)', 'ADDRESS_2', 'PROVINCE', 'PROVINCE_CODE', 'LATITUDE', 'LONGITUDE',
                'ZIP', 'COUNTRY_CODE', 'CITY', 'LIFETIME_DURATION', 'CUSTOMER_ID'], axis=1, inplace=True)

In [ ]:
df.head(10)

In [ ]:
for i in df.columns:

    if i.startswith('Channel') or i.startswith('Event'):
        df[i].fillna(0, inplace=True)
    elif i == 'Sessions_minutes':
        df[i].fillna(df[i].mean(), inplace=True)
    elif i == 'Sessions_pages':
        df[i].fillna(df[df['Sessions_count']==1][i].mean(), inplace=True)
    elif i == 'Sessions_first_order':
        df[i].fillna(df[df['Sessions_count']==1][i].mean(), inplace=True)
    elif i == 'Sessions_count':
        df[i].fillna(1, inplace=True)

# for i in df.columns:
#     print(i, ': ', sum(df[i].isna()))
    
df.dropna(inplace=True)

In [ ]:
c = df.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
so

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm') # This line plots nicely visualized matrix of correlation

In [163]:
df['ACCEPTS_MARKETING'] = df['ACCEPTS_MARKETING']*1
df.columns

Index(['ACCEPTS_MARKETING', 'ORDERS_COUNT', 'TOTAL_SPENT', 'USER_ID', 'ID_y',
       'Channel_bing_ads', 'Channel_bing_organic',
       'Channel_duckduckgo_organic', 'Channel_ecosia_organic', 'Channel_email',
       'Channel_facebook_ads', 'Channel_facebook_organic',
       'Channel_google_ads', 'Channel_google_organic', 'Channel_inspiration',
       'Channel_instagram_organic', 'Channel_internal_traffic',
       'Channel_other_cpc_ads', 'Channel_referral', 'Channel_social',
       'Event_Accordion_Clicked', 'Event_Amazon_Button_Clicked',
       'Event_Announcement_Clicked', 'Event_Blog_Post_Clicked',
       'Event_Blog_Post_Viewed', 'Event_Blogpage_Hero_Clicked',
       'Event_Carousel_Clicked', 'Event_Cart_Viewed',
       'Event_Category_Page_Viewed', 'Event_Checkout_Started',
       'Event_Checkout_Step_Viewed', 'Event_Collection_Anchor_Clicked',
       'Event_Collection_Clicked', 'Event_Collection_Gender_Selected',
       'Event_Collection_Page_Viewed', 'Event_Color_Selected',
    

In [ ]:
df_all = pd.get_dummies(df, columns=['COUNTRY'])
df_all.drop(['ID_y', 'USER_ID'], axis=1, inplace=True)

In [ ]:
Xstd_all = StandardScaler().fit_transform(df_all)
pca_all = PCA(n_components=10)
components_all = pca_all.fit_transform(Xstd_all)

In [ ]:
features = range(pca_all.n_components_)
plt.bar(features, pca_all.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)

In [ ]:
# Save components to a DataFrame
PCA_components_all = pd.DataFrame(components_all)

In [ ]:
plt.scatter(PCA_components_all[0], PCA_components_all[1], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
ks = range(1, 10)
inertias_all = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model_all = KMeans(n_clusters=k)
    
    # Fit model to samples
    model_all.fit(PCA_components_all.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias_all.append(model_all.inertia_)
    
plt.plot(ks, inertias_all, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
model_all_chosen = KMeans(n_clusters=4)
model_all_chosen.fit(PCA_components.iloc[:,:3])

In [ ]:
df_all_result = pd.concat([df.reset_index(drop=True), pd.DataFrame(PCA_components_all.iloc[:,:3])], axis=1)

In [ ]:
df_all_result['segment'] = model_all.labels_

In [ ]:
df_all.columns[1:40]

## SECOND APPROACH

In [ ]:
# for i in df.columns:

#     if i.startswith('Event'):
#         print(i,': ', sum(RobustScaler().fit_transform(df[[i]])) / len(df[i]))

In [157]:
df_merge = df.copy()

In [158]:
for i in df_merge['COUNTRY']:
    if i in ['Belgium', 'Luxembourg']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'BE_LUX')
    elif i in ['Netherlands', 'Netherlands Antilles']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'NL_ANG')
    elif i in ['United Kingdom', 'Ireland', 'Guernsey', 'Jersey']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'UK_IR')
    elif i in ['Finland', 'Denmark', 'Sweden', 'Norway', 'Iceland']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'North_EU')
    elif i in ['Germany', 'Austria']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'DE_AT')
    elif i in ['Croatia', 'Hungary', 'Slovakia', 'Poland', 'Romania', 'Ukraine', 'Bulgaria', 'Lithuania', 'Latvia', 'Estonia',  'Bosnia', 'Russia', 'Czech Republic', 'Slovenia', 'North Macedonia']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'East_EU')
    elif i in ['France', 'Spain', 'Malta', 'Portugal', 'Greece', 'Italy', 'Reunion', 'Monaco']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'South-Med_EU')
    elif i in ['Trinidad and Tobago', 'New Zealand', 'Philippines', "Côte d'Ivoire", 'Oman', 'Solomon Islands','Turkey', 'United Arab Emirates', 'United States', 'Australia', 'Brunei', 'Cyprus']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'Non_EU' )
    elif i in ['Switzerland', 'Liechtenstein']:
        df_merge['COUNTRY'] = df_merge['COUNTRY'].replace(i, 'CH_LI')

In [159]:
df_merge['COUNTRY'].value_counts()

NL_ANG          37443
DE_AT           26318
BE_LUX          14699
UK_IR            6158
CH_LI            1180
South-Med_EU     1020
North_EU          413
East_EU           182
Non_EU             55
Name: COUNTRY, dtype: int64

In [160]:
df_merge['Event_Blog'] = df_merge['Event_Blog_Post_Clicked'] + df_merge['Event_Blog_Post_Viewed'] +  df_merge['Event_Blogpage_Hero_Clicked']
df_merge['Event_Email'] = df_merge['Event_Email_Capture_Closed'] +df_merge['Event_Email_Capture_Submitted'] + df_merge['Event_Email_Capture_Viewed']
df_merge['Event_Collection'] = df_merge['Event_Collection_Anchor_Clicked'] + df_merge['Event_Collection_Clicked'] + df_merge['Event_Collection_Gender_Selected'] + df_merge['Event_Collection_Page_Viewed'] + df_merge['Event_Interaction_with_Collection_Gender_Filter']
df_merge['Event_Footer'] = df_merge['Event_Footer_Clicked'] +df_merge ['Event_Footer_Viewed']
df_merge['Event_Review'] = df_merge['Event_Homepage_Review_Clicked'] + df_merge['Event_Review_Page_Viewed'] + df_merge['Event_Load_More_Reviews'] + df_merge['Event_Review_Anchor_Clicked']
df_merge['Event_Accordion'] = df_merge['Event_Accordion_Clicked'] + df_merge['Event_Interaction_with_Product_Accordion']
df_merge['Event_Nav'] = df_merge['Event_Nav_Clicked'] + df_merge['Event_Nav_Closed'] + df_merge['Event_Nav_Opened']
df_merge['Event_Product'] = df_merge['Event_Product_Added'] + df_merge['Event_Product_Clicked'] + df_merge['Event_Product_Info_Clicked'] + df_merge['Event_Product_List_Viewed'] + df_merge['Event_Product_Removed'] + df_merge['Event_Product_Viewed'] + df_merge['Event_Product_Zoom_Clicked']
df_merge['Event_Search'] = df_merge['Event_Search_Clicked'] + df_merge['Event_Search_Result_Clicked']
df_merge['Event_Cart_Checkout'] = df_merge['Event_Cart_Viewed'] + df_merge['Event_Checkout_Started'] + df_merge['Event_Checkout_Step_Viewed']
df_merge['Event_Carousel'] = df_merge['Event_Carousel_Clicked']

In [161]:
df_merge['Channel_Organic'] = df_merge['Channel_bing_organic'] + df_merge['Channel_duckduckgo_organic'] + df_merge['Channel_instagram_organic'] + df_merge['Channel_ecosia_organic'] + df_merge['Channel_facebook_organic'] + df_merge['Channel_google_organic']
df_merge['Channel_Social'] = df_merge['Channel_inspiration'] + df_merge['Channel_referral'] + df_merge['Channel_social']
df_merge['Channel_Ads'] = df_merge['Channel_facebook_ads'] + df_merge['Channel_google_ads'] + df_merge['Channel_other_cpc_ads'] + df_merge['Channel_bing_ads']
df_merge['Channel_Internal_Traffic'] = df_merge['Channel_internal_traffic']
df_merge['Channel_Email'] = df_merge['Channel_email']

In [167]:
df_merge.columns

Index(['ACCEPTS_MARKETING', 'ORDERS_COUNT', 'TOTAL_SPENT', 'USER_ID', 'ID_y',
       'Channel_bing_ads', 'Channel_bing_organic',
       'Channel_duckduckgo_organic', 'Channel_ecosia_organic', 'Channel_email',
       'Channel_facebook_ads', 'Channel_facebook_organic',
       'Channel_google_ads', 'Channel_google_organic', 'Channel_inspiration',
       'Channel_instagram_organic', 'Channel_internal_traffic',
       'Channel_other_cpc_ads', 'Channel_referral', 'Channel_social',
       'Event_Accordion_Clicked', 'Event_Amazon_Button_Clicked',
       'Event_Announcement_Clicked', 'Event_Blog_Post_Clicked',
       'Event_Blog_Post_Viewed', 'Event_Blogpage_Hero_Clicked',
       'Event_Carousel_Clicked', 'Event_Cart_Viewed',
       'Event_Category_Page_Viewed', 'Event_Checkout_Started',
       'Event_Checkout_Step_Viewed', 'Event_Collection_Anchor_Clicked',
       'Event_Collection_Clicked', 'Event_Collection_Gender_Selected',
       'Event_Collection_Page_Viewed', 'Event_Color_Selected',
    